import pandas as pd
# ^^^ pyforest auto-imports - don't write above this line
Notebook to prepare the data

In [1]:
import numpy as np
import pandas as pd
from glob import glob
from os.path import join as opj
from nilearn import image
from nilearn.masking import apply_mask

In [2]:
response_vars = ["mavgccaf_chgPerYr", "Site_of_Fastest_IMT_Max_Progr"]
YY_df = pd.read_csv("../data/Thomas_Javi_ML_Share_PIP_Reappraisal_IMT_Longitudinal_02_NOV_2020.csv", na_values=' ').loc[:, ["ePrime.id"] + response_vars]

In [3]:
final_subjects_used = YY_df.dropna().iloc[:,0].to_list()
print("the number of final subjects for prediction used are then = %i" % len(final_subjects_used))

the number of final subjects for prediction used are then = 145


In [4]:
# This is the mask that we are going to use to keep only grey matter voxels
mask = "../data/reslicedER_grey25grey25.nii"

In [5]:
# As a sanity check, see that contrasts and mask have the same dimension
print(image.load_img(mask).shape)
print(image.load_img(opj("../data/", "contrasts", "sub-%s" % final_subjects_used[0], "con_0001.nii")).shape)

(79, 95, 79)
(79, 95, 79)


# data main analyses

In [7]:
imgs_0001 = [opj("../data/", "contrasts", "sub-%s" % sub_id, "con_0001.nii") for sub_id in final_subjects_used]
imgs_0002 = [opj("../data/", "contrasts", "sub-%s" % sub_id, "con_0002.nii") for sub_id in final_subjects_used]

X_1 = apply_mask(imgs_0001, mask)
X_2 = apply_mask(imgs_0002, mask)

print(X_1.shape)
print(X_2.shape)

# Save input data
np.savez("../data/input_data.npz", look_neg_look_neut =  X_1, reg_neg_look_neg = X_2)

# Save target data for main analyses
pd.merge(pd.DataFrame({'ePrime.id':final_subjects_used}), 
         pd.read_csv("../data/Thomas_Javi_ML_Share_PIP_Reappraisal_IMT_Longitudinal_02_NOV_2020.csv", 
                     na_values=' '), 
         on = 'ePrime.id').to_csv("../data/target_data.csv", index=False)
pd.read_csv("../data/target_data.csv").shape

(145, 207584)
(145, 207584)


(145, 21)

# data sensitivity analyses

In [8]:
to_discard = np.loadtxt("../data/subjects_discard_sensitivity.txt", dtype=int)
print("number of subjects to drop for sensitivity analysys = ", len(to_discard))

subject_sensitivity = list(set(final_subjects_used).difference(set(to_discard)))
print("number of subjects to run the sensitivity analyses = ", len(subject_sensitivity))

number of subjects to drop for sensitivity analysys =  8
number of subjects to run the sensitivity analyses =  137


In [9]:
imgs_0001 = [opj("../data/", "contrasts", "sub-%s" % sub_id, "con_0001.nii") for sub_id in subject_sensitivity]
imgs_0002 = [opj("../data/", "contrasts", "sub-%s" % sub_id, "con_0002.nii") for sub_id in subject_sensitivity]

X_1 = apply_mask(imgs_0001, mask)
X_2 = apply_mask(imgs_0002, mask)

print(X_1.shape)
print(X_2.shape)

# Save input data for sensitivity analyses
np.savez("../data/input_data_sensitivity.npz", 
         look_neg_look_neut =  X_1, 
         reg_neg_look_neg = X_2)

# Save target data for sensitivity analyses
pd.merge(pd.DataFrame({'ePrime.id':subject_sensitivity}), 
         pd.read_csv("../data/Thomas_Javi_ML_Share_PIP_Reappraisal_IMT_Longitudinal_02_NOV_2020.csv", na_values=' '), 
         on = 'ePrime.id').to_csv("../data/target_data_sensitivity.csv", index=False)
pd.read_csv("../data/target_data_sensitivity.csv").shape

(137, 207584)
(137, 207584)


(137, 21)